In [ ]:
%load_ext autoreload
%autoreload 2
%matplotlib widget

# there are various runtime warnings generated by the presence of infinite dt values
# in the analysis. this is not a bug. ignore these warnings ...
import warnings
warnings.simplefilter(action='ignore', category=RuntimeWarning)

import os

import numpy as np
import pandas as pd
import geopandas as gpd
import xarray as xr
from scipy.stats import gaussian_kde
from scipy.stats import t
from itertools import product

import matplotlib.pyplot as plt

from cartopy import crs as ccrs, feature as cfeature

from station_analysis import load_and_process_station_data
from aggregate_and_visualize import (
    dt_table,
    o2c_scatter_map,
    o2c_scatter_plot,
    hist_violin,
    plot_change_with_slr,
    MulticolorPatch,
    MulticolorPatchHandler,
)

## Load global analysis and metadata

First configure the quantities to use in the aggregations and figures.

In [ ]:
central_est_type = "median"  # mean or median
chronic_freq = 26  # 20, 26, or 50 days per year
scenario = "Int"  # Low, IntLow, Int, IntHigh, High
significance_level = 95  # 80, 90, 95, or 99

# colors = ["#63b4d1", "#db3a34", "#ece4d5", "#BD93D8"]
# colors = ["#63b4d1", "#710627", "#ece4d5", "#B6A1BA"]
# colors = ["#63b4d1", "#db3a34", "#dedede", "#87C38F"]
colors = ["#56B4E9", "#F3EA68", "#dddddd", "#CC79A7", "#E69F00", "#D55E00", "#009E73"]

Load the analysis.

In [ ]:
output_dir = "./output"

analysis = pd.read_csv(f"{output_dir}/global_analysis.csv", index_col=0)
analysis = gpd.GeoDataFrame(
    analysis, geometry=gpd.points_from_xy(analysis.lon, analysis.lat)
)

analysis

## Load results of geographic sampling experiment

In [ ]:
# for binary global classifications

io_median_diff_percentiles = pd.read_csv(
    f"{output_dir}/group_median_differences_percentiles.csv", index_col=0
)
io_median_diff_percentiles.columns = [
    int(c) for c in io_median_diff_percentiles.columns
]
io_median_diff_ensemble = pd.read_csv(
    f"{output_dir}/group_median_differences_ensemble.csv", index_col=0
)

io_mean_diff_percentiles = pd.read_csv(
    f"{output_dir}/group_mean_differences_percentiles.csv", index_col=0
)
io_mean_diff_percentiles.columns = [int(c) for c in io_mean_diff_percentiles.columns]
io_mean_diff_ensemble = pd.read_csv(
    f"{output_dir}/group_mean_differences_ensemble.csv", index_col=0
)

# for subsetted groups

io_exclusion_median_diff_percentiles = pd.read_csv(
    f"{output_dir}/group_exclusion_median_differences_percentiles.csv", index_col=0
)
io_exclusion_median_diff_percentiles.columns = [
    int(c) for c in io_exclusion_median_diff_percentiles.columns
]
io_exclusion_median_diff_ensemble = pd.read_csv(
    f"{output_dir}/group_exclusion_median_differences_ensemble.csv", index_col=0
)

io_exclusion_mean_diff_percentiles = pd.read_csv(
    f"{output_dir}/group_exclusion_mean_differences_percentiles.csv", index_col=0
)
io_exclusion_mean_diff_percentiles.columns = [
    int(c) for c in io_exclusion_mean_diff_percentiles.columns
]
io_exclusion_mean_diff_ensemble = pd.read_csv(
    f"{output_dir}/group_exclusion_mean_differences_ensemble.csv", index_col=0
)

## Differences between geographic groups of TG locations

### Define Groups

In [ ]:
# load list of island stations (also includes designation as colonized or not)
islands = pd.read_csv("./data/islands.csv")

not_island_ids = [uid for uid in analysis.index if uid not in islands.uhid]
not_island_df = analysis.loc[not_island_ids, ["name", "country", "lon", "lat"]]
not_island_df.index.name = "uhid"
not_island_df = not_island_df.sort_values(by="country")
not_island_df.to_csv("./data/not_islands.csv", index=True)

In [ ]:
groups = {
    "all": analysis.hdi > -1000,
    # global south and global north
    "globalN": analysis.hdi >= 0.8,
    "globalS": analysis.hdi < 0.8,
    # low latitude (< 30) and high latitude (> 30)
    "highLat": np.abs(analysis.lat) > 30,
    "lowLat": np.abs(analysis.lat) <= 30,
    # East and West Hemispheres
    "westHemi": (analysis.lon < 30) | (analysis.lon >= 180),
    "eastHemi": (analysis.lon >= 30) & (analysis.lon < 180),
    # islands and continents
    "island": analysis.index.isin(islands.uhid),
    "continent": ~analysis.index.isin(islands.uhid),
    "colonized": analysis.index.isin(islands.uhid.loc[islands.colonized]),
}

groups = {
    **groups,
    **{
        # lat bands east and west
        "highLat_westHemi": groups["highLat"] & groups["westHemi"],
        "highLat_eastHemi": groups["highLat"] & ~groups["westHemi"],
        "lowLat_westHemi": groups["lowLat"] & groups["westHemi"],
        "lowLat_eastHemi": groups["lowLat"] & ~groups["westHemi"],
        # islands in the global south
        "globalS_island": groups["island"] & groups["globalS"],
        "globalS_continent": groups["continent"] & groups["globalS"],
        # global north high-latitude
        "globalN_highLat": groups["globalN"] & groups["highLat"],
        # global north colonized
        "globalN_colonized": groups["globalN"] & groups["colonized"],
        # global north excluding colonized (i.e., domestic)
        "globalN_domestic": groups["globalN"] & ~groups["colonized"],
        # continental in the global north
        "globalN_continent": groups["globalN"] & groups["continent"],
        # global north west hemisphere
        "globalN_westHemi": groups["globalN"] & groups["westHemi"],
    },
}

groups = {
    **groups,
    **{
        "globalN_westHemi_continent": groups["globalN_westHemi"] & groups["continent"],
        "globalN_westHemi_highLat": groups["globalN_westHemi"] & groups["highLat"],
    },
}

groups = {
    **groups,
    **{
        "globalN_domestic_westHemi": groups["globalN_domestic"] & groups["westHemi"],
        "globalN_domestic_highLat": groups["globalN_domestic"] & groups["highLat"],
        "globalN_domestic_westHemi_highLat": (
            groups["globalN_domestic"] & groups["highLat_westHemi"]
        ),
    },
}

groups = {
    **groups,
    **{
        "globalN_domestic_NAEU": (
            groups["globalN_domestic_westHemi"] & (analysis.lat > 22)
        ),
        "globalN_domestic_westHemiOther": (
            groups["globalN_domestic_westHemi"] & (analysis.lat <= 22)
        ),
    },
}


# convert to numpy boolean to avoid issues with pandas indices
groups = {
    g: groups[g].values if type(groups[g]) is not np.ndarray else groups[g]
    for g in groups
}

In [ ]:
pct_ll_in_isl = (
    100 * (groups["lowLat"] & groups["island"]).sum() / groups["island"].sum()
)
pct_isl_in_ll = (
    100 * (groups["lowLat"] & groups["island"]).sum() / groups["lowLat"].sum()
)
print(f"{pct_ll_in_isl:.1f}% of islands are low latitude")
print(f"{pct_isl_in_ll:.1f}% of low latitude locations are islands")

In [ ]:
# the difference in dt global medians between the two start years for the chosen central estimate, scenario, and chronic frequency
dt_50 = analysis[f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2050"].median()
dt_20 = analysis[f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"].median()
dt_diff = dt_20 - dt_50
print(f"dt beginning 2020: {dt_20:.2f} years")
print(f"dt beginning 2050: {dt_50:.2f} years")
print(f"dt difference (2020 minus 2050): {dt_diff:.2f} years")

In [ ]:
# add a couple of groups to the analysis dataframe for making manuscript table
analysis["island"] = groups["island"]
analysis["domestic"] = ~groups["colonized"]

### Explore groups

In [ ]:
fig, ax = plt.subplots(
    3,
    1,
    figsize=(6, 8),
    subplot_kw=dict(projection=ccrs.Mollweide(central_longitude=210)),
)

# for n, g in enumerate(["lowLat", "globalS", "island"]):
for n, g in enumerate(["globalS", "globalN_colonized", "globalN_domestic_NAEU"]):

    ax[n].add_feature(cfeature.LAND.with_scale("110m"), color="gray")
    ax[n].gridlines(linewidth=0.5, color="k", linestyle=":")
    co = ax[n].scatter(
        x=analysis.lon,
        y=analysis.lat,
        c=colors[0],
        edgecolors="k",
        linewidths=0.5,
        s=50,
        transform=ccrs.PlateCarree(),
    )
    co = ax[n].scatter(
        x=analysis.lon.loc[groups[g]],
        y=analysis.lat.loc[groups[g]],
        c=colors[1],
        edgecolors="k",
        linewidths=0.5,
        s=50,
        transform=ccrs.PlateCarree(),
    )
    ax[n].set_global()

plt.tight_layout()

### Calculate median and other percentiles of each group

In [ ]:
# first, a few global HDI statistics based on countries, not TGs
national_values = {
    "Global South HDI": np.median(
        analysis.hdi.loc[groups["globalS"]].dropna().unique()
    ),
    "Domestic NA/EU HDI": np.median(
        analysis.hdi.loc[groups["globalN_domestic_NAEU"]].dropna().unique()
    ),
    "Domestic WH Not NA/EU HDI": np.median(
        analysis.hdi.loc[groups["globalN_domestic_westHemiOther"]].dropna().unique()
    ),
}
pd.Series(national_values)

In [ ]:
# isolate numeric columns in the analysis dataframe
analysis_data = analysis[
    [
        c
        for c in analysis.columns
        if c
        not in [
            "name",
            "country",
            "lat",
            "lon",
            "geometry",
            "island",
            "domestic",
        ]
    ]
]

percentiles = [5, 17, 50, 83, 95]
group_percentiles = pd.DataFrame(
    index=[g for g in groups], columns=analysis_data.columns
)
for g in groups:
    this_group_percentiles = pd.DataFrame(
        [analysis_data.loc[groups[g]].quantile(p / 100, axis=0) for p in percentiles],
        index=percentiles,
    ).round(4)
    group_percentiles.loc[g] = pd.Series(
        {c: this_group_percentiles[c].to_list() for c in analysis_data.columns}
    )
group_counts = pd.Series({g: groups[g].sum() for g in groups})
group_counts.name = "N"
group_percentiles = pd.concat([group_counts, group_percentiles], axis=1)
group_percentiles[
    [
        "N",
        "hdi",
        f"dh_{central_est_type}_{chronic_freq}_days",
        f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020",
        f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2050",
    ]
]

### Calculate median group differences and probability that each could occur by chance

In [ ]:
io_diff_ensemble = pd.read_csv(
    f"{output_dir}/group_{central_est_type}_differences_ensemble.csv"
)
io_exclusion_diff_ensemble = pd.read_csv(
    f"{output_dir}/group_exclusion_{central_est_type}_differences_ensemble.csv"
)


def prob_group_diff_occur_chance(group_1, group_2, data, diff_ensemble):
    diff = (
        (data.loc[group_2].median(axis=0) - data.loc[group_1].median(axis=0))
        .abs()
        .round(4)
    )
    prob = (diff_ensemble > diff).sum(axis=0) / diff_ensemble.index.size
    return pd.Series(
        [(d, p) for d, p in zip(diff.values, prob.values)], index=diff.index
    )


groups_to_compare = dict(
    globalN_vs_globalS=["globalN", "globalS"],
    highLat_vs_lowLat=["highLat", "lowLat"],
    continent_vs_island=["continent", "island"],
    HLWH_vs_HLEH=["highLat_westHemi", "highLat_eastHemi"],
    LLWH_vs_LLEH=["lowLat_westHemi", "lowLat_eastHemi"],
    GS_vs_DGN=["globalS", "globalN_domestic"],
    GS_vs_DGN_WH=["globalS", "globalN_domestic_westHemi"],
    GS_vs_DGN_NAEU=["globalS", "globalN_domestic_NAEU"],
    # GS_vs_GNHL=["globalS", "globalN_highLat"],
    # GS_vs_GNWHHL=["globalS", "globalN_westHemi_highLat"],
    # GS_vs_GNWHCN=["globalS", "globalN_westHemi_continent"],
)
all_gauge_comps = ["globalN_vs_globalS", "highLat_vs_lowLat", "continent_vs_island"]

group_diff_with_prob_chance = pd.DataFrame(
    [
        prob_group_diff_occur_chance(
            groups[groups_to_compare[gc][0]],
            groups[groups_to_compare[gc][1]],
            analysis_data,
            io_diff_ensemble if gc in all_gauge_comps else io_exclusion_diff_ensemble,
        )
        for gc in groups_to_compare
    ],
    index=[gc for gc in groups_to_compare],
)
group_diff_with_prob_chance[
    [
        f"dh_{central_est_type}_{chronic_freq}_days",
        f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020",
        f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2050",
        f"res_dymx_std",
        f"slr_total_{scenario}_2020_2050",
        f"slr_ocean_dyn_{scenario}_2020_2050",
        f"slr_massdef_{scenario}_2020_2050",
        f"slr_vlm_{scenario}_2020_2050",
        f"slr_ais_{scenario}_2020_2050",
        f"slr_gis_{scenario}_2020_2050",
        f"slr_glaciers_{scenario}_2020_2050",
        f"slr_landwater_{scenario}_2020_2050",
    ]
]

## Make station table for manuscript

In [ ]:
columns = dict(
    uhid=("Station Metadata", "UH ID"),
    name=("Station Metadata", "Name"),
    country=("Station Metadata", "Country"),
    lon=("Station Metadata", "Longitude"),
    lat=("Station Metadata", "Latitude"),
    n_good_years=("Station Metadata", "Valid Years"),
    hdi=("Classifications", "HDI"),
    island=("Classifications", "Island"),
    domestic=("Classifications", "Domestic"),
    dh_median_26_days=("Transitions", "∆h (cm)"),
    dt_median_26_days_Int_2020=("Transitions", "∆t (years)"),
    res_hf_dymx_std=("Process Proxies (cm)", "Storminess"),
    res_momn_q75_std=("Process Proxies (cm)", "MSL Variability"),
    tide_dymx_std=("Process Proxies (cm)", "High-Tide Modulation"),
    slr_total_Int_2020_2050=("Relative SLR (cm)", "Total"),
    slr_ocean_dyn_Int_2020_2050=("Relative SLR (cm)", "Sterodynamic"),
    slr_massdef_Int_2020_2050=("Relative SLR (cm)", "Mass and Deformation"),
    slr_vlm_Int_2020_2050=("Relative SLR (cm)", "Vertical Land Motion"),
)
station_table = analysis.loc[:, [c for c in columns][1:]]
station_table = station_table.reset_index()
station_table = station_table.sort_values(by=["country", "name"])
station_table.columns = pd.MultiIndex.from_tuples([columns[c] for c in columns])
station_table

In [ ]:
# Create a Pandas Excel writer using XlsxWriter as the engine
with pd.ExcelWriter(
    "./output/supplementary_tables_unformatted.xlsx", engine="xlsxwriter"
) as writer:
    # Write the DataFrame to the Excel file
    station_table.to_excel(writer, sheet_name="Stations")

    # Access the XlsxWriter workbook and worksheet objects
    workbook = writer.book
    worksheet = writer.sheets["Stations"]

## Figures

### $\Delta h$ schematic

In [ ]:
stations = [
    # dict(uhid=57, figure_name="Honolulu, HI"),
    dict(uhid=5, figure_name="Majuro, MI"),
    dict(uhid=264, figure_name="Atlantic City, USA"),
]

run_schematic_analysis = False

schematic_locations_fn = "./output/schematic_locations.csv"
if os.path.exists(schematic_locations_fn):
    locations = pd.read_csv(
        schematic_locations_fn,
        dtype=dict(
            station_country_code=int,
            uhslc_id=int,
        ),
        index_col="uhslc_id",
    )
else:
    locations = None

schematic_dymxdt_fn = "./output/schematic_daily_max_dt.csv"
if os.path.exists(schematic_dymxdt_fn):
    daily_max_dt = pd.read_csv(
        schematic_dymxdt_fn,
        index_col="time",
        parse_dates=True,
    )
    daily_max_dt.columns = daily_max_dt.columns.astype(int)
else:
    daily_max_dt = None

for uhid in [s["uhid"] for s in stations]:
    if locations is None or daily_max_dt is None:
        run_schematic_analysis = True
        break
    if uhid not in locations.index or uhid not in daily_max_dt.columns:
        run_schematic_analysis = True

fn = "./data/tide_gauge_locations.csv"

# if the schematic analysis already does not exist, do the analysis
if run_schematic_analysis:

    # load the metadata file if it already exists
    locations = pd.read_csv(
        fn,
        dtype=dict(
            station_country_code=int,
            uhslc_id=int,
        ),
        index_col="uhslc_id",
    )

    locations = locations.loc[[s["uhid"] for s in stations]]
    locations.station_name = [s["figure_name"] for s in stations]
    locations[["h01", "h26"]] = None

    min_hours_per_day = 20
    min_days_per_year = 320
    min_years_for_inclusion = 9

    daily_max_dt = dict()

    def get_detrended_tg_data(tg):
        hsl, hsl_trnd, _, _ = load_and_process_station_data(
            tg,
            min_hours_per_day,
            min_days_per_year,
            min_years_for_inclusion,
            tide_prd_dir="./data/tide_predictions/",
        )

        # return detrended hourly values
        return hsl - hsl_trnd

    for uhid, tg in locations.iterrows():

        hsldt = get_detrended_tg_data(tg)

        # get daily max of detrended hourly values
        dmxdt = hsldt.groupby(pd.Grouper(freq="D")).apply(
            lambda x: x.mean() if (x.dropna().count() > min_hours_per_day) else None
        )

        # calculate high and low
        high = np.floor(np.max(dmxdt))
        low = np.floor(0.5 * np.std(dmxdt))  # halfstd

        # define range of thresholds
        thresholds = np.arange(low, high + 0.1, 0.1)

        # get number of daily max exceedances in each met year over each threshold
        dmxdt_gy = dmxdt.groupby(dmxdt.index.year)
        Nxpy = pd.concat(
            [dmxdt_gy.apply(lambda x: (x > thrsh).sum()) for thrsh in thresholds],
            axis=1,
        )
        Nxpy.columns = thresholds

        # calculate the average number of daily max exceedances per met year for each threshold
        Nxpy_mn = Nxpy.median(axis=0)

        h01 = Nxpy_mn.loc[Nxpy_mn >= 1].idxmin()
        h26 = Nxpy_mn.loc[Nxpy_mn >= 26].idxmin()

        daily_max_dt[uhid] = dmxdt
        locations.loc[uhid, "h01"] = h01
        locations.loc[uhid, "h26"] = h26

    daily_max_dt = pd.DataFrame(daily_max_dt)
    daily_max_dt.to_csv(schematic_dymxdt_fn, index=True)

    locations.to_csv(schematic_locations_fn, index=True)

locations

In [ ]:
fig1, ax1 = plt.subplots(figsize=(10, 8))

x_offset = 11000
y_offset = [0, -160]  # -145]
dst_fact = [5e4, 10e4]
name_offset = [-25, -25]
scale_bottom = -275
for n, (uhid, tg) in enumerate(locations.iterrows()):
    dmxdt = daily_max_dt[uhid]
    tg_name, h01, h26 = locations.loc[uhid][["station_name", "h01", "h26"]].to_list()

    dmxdt_rng = dmxdt.loc["1998.5":"2022.5"].copy()
    dmxdt_rng.index = range(dmxdt_rng.index.size)

    dst_vals = dmxdt_rng.dropna().values
    kde = gaussian_kde(dst_vals)
    dst_x = np.linspace(min(dst_vals), max(dst_vals), 1000)
    dst = pd.Series(kde(dst_x), index=dst_x)

    plot_change_with_slr(
        ax=ax1,
        name=tg_name,
        dymx=dmxdt_rng,
        dst=dst,
        dst_fact=dst_fact[n],
        colors=[colors[n] for n in [0, 1, 0]],
        offsets=[x_offset, y_offset[n]],
        thrsh=h01,
        slr=h01 - h26,
        name_offset=name_offset[n],
        scale_bottom=scale_bottom,
        labels=True if n == 0 else False,
    )

ax1.axis("off")
ax1.set_xlim([-4000, dmxdt_rng.index[-1] + x_offset + 4000])
ax1.set_ylim([scale_bottom - 5, 50])

plt.legend(
    loc="lower right", bbox_to_anchor=(0.92, -0.015), frameon=False, borderpad=0, ncol=2
)
plt.tight_layout()

fig1.savefig("./figures/manuscript/dh_schematic.png", dpi=300)
fig1.savefig("./figures/manuscript/dh_schematic.pdf", dpi=300)

### Table of $\Delta t$ values

In [ ]:
table_groups = dict(
    all="Global",
    lowLat="Low Latitude",
    highLat="High Latitude",
    globalS="Global South",
    globalN="Global North",
    globalN_domestic_NAEU="Domestic N.A./Europe",
)
table_years = [2020, 2040, 2060]
table_scenarios = dict(IntLow="Int. Low", Int="Intermediate", IntHigh="Int. High")
table_columns = [c for c in product([s for s in table_scenarios], table_years)]
table_quantities = [f"dt_median_26_days_{c[0]}_{c[1]}" for c in table_columns]


table_dt = group_percentiles.loc[[g for g in table_groups], table_quantities]
table_dt = table_dt.map(lambda x: x[2] if len(x) > 2 else None).round().astype(int)
table_dt.columns = table_columns

fig = dt_table(table_dt, table_groups, table_scenarios)

fig.savefig("./figures/manuscript/dt_table.png", dpi=300)
fig.savefig("./figures/manuscript/dt_table.pdf", dpi=300)

### Scatter maps of $\Delta h$ and $\Delta t$

In [ ]:
# --------------------------------------------------------------------
# make the figure and subplots

fig, (ax1, ax2, ax3) = plt.subplots(
    3,
    1,
    figsize=(8, 10),
    subplot_kw=dict(projection=ccrs.Mollweide(central_longitude=210)),
)

# --------------------------------------------------------------------
# delta h

c_sorted = analysis[f"dh_{central_est_type}_{chronic_freq}_days"].sort_values(
    ascending=True
)
o2c_scatter_map(
    fig=fig,
    ax=ax1,
    x=analysis.lon.loc[c_sorted.index],
    y=analysis.lat.loc[c_sorted.index],
    c=c_sorted,
    squares=analysis.hdi.loc[c_sorted.index] < 0.8,
    vmin=5,
    vmax=30,
    alpha=1,
    splab="a",
    title="∆h",
    cbar_label="cm",
)

# --------------------------------------------------------------------
# delta t beginning 2020

c_sorted = analysis[
    f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"
].sort_values(ascending=True)
o2c_scatter_map(
    fig=fig,
    ax=ax2,
    x=analysis.lon.loc[c_sorted.index],
    y=analysis.lat.loc[c_sorted.index],
    c=c_sorted,
    squares=analysis.hdi.loc[c_sorted.index] < 0.8,
    vmin=5,
    vmax=50,
    alpha=1,
    splab="b",
    title="∆t beginning in 2020",
    cbar_label="years",
)

# --------------------------------------------------------------------
# Delta t beginning 2050

c_sorted = analysis[
    f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2050"
].sort_values(ascending=True)
o2c_scatter_map(
    fig=fig,
    ax=ax3,
    x=analysis.lon.loc[c_sorted.index],
    y=analysis.lat.loc[c_sorted.index],
    c=c_sorted,
    squares=analysis.hdi.loc[c_sorted.index] < 0.8,
    vmin=5,
    vmax=50,
    alpha=1,
    splab="c",
    title="∆t beginning in 2050",
    cbar_label="years",
)

# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

# fig.savefig("./figures/manuscript/dh_dt_scatter_maps.png", dpi=300)
# fig.savefig("./figures/manuscript/dh_dt_scatter_maps.pdf", dpi=300)

In [ ]:
# --------------------------------------------------------------------
# make the figure and subplots

fig, (ax1, ax2) = plt.subplots(
    2,
    1,
    figsize=(8, 7),
    subplot_kw=dict(projection=ccrs.Mollweide(central_longitude=210)),
)

# --------------------------------------------------------------------
# delta h

c_sorted = analysis[f"dh_{central_est_type}_{chronic_freq}_days"].sort_values(
    ascending=True
)
o2c_scatter_map(
    fig=fig,
    ax=ax1,
    x=analysis.lon.loc[c_sorted.index],
    y=analysis.lat.loc[c_sorted.index],
    c=c_sorted,
    squares=analysis.hdi.loc[c_sorted.index] < 0.8,
    vmin=5,
    vmax=30,
    alpha=1,
    splab="a",
    title="∆h",
    cbar_label="cm",
)

# --------------------------------------------------------------------
# delta t beginning 2020

c_sorted = analysis[
    f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"
].sort_values(ascending=True)
o2c_scatter_map(
    fig=fig,
    ax=ax2,
    x=analysis.lon.loc[c_sorted.index],
    y=analysis.lat.loc[c_sorted.index],
    c=c_sorted,
    squares=analysis.hdi.loc[c_sorted.index] < 0.8,
    vmin=5,
    vmax=50,
    alpha=1,
    splab="b",
    title="∆t",
    cbar_label="years",
)

# --------------------------------------------------------------------

leglab = ["Global South", "Global North"]
# make dummy square and circle plots for the legend
att = dict(c="lightgray", ls="none", lw=0.5, ms=8, mec="k", mew=0.5)
l1 = ax1.plot(0, 1e7, marker="s", label=leglab[0], **att)[0]
l2 = ax1.plot(0, 1e7, marker="o", label=leglab[1], **att)[0]
fig.legend(
    [l1, l2],
    leglab,
    ncols=1,
    loc="center left",
    bbox_to_anchor=(0.73, 0.485),
    frameon=False,
    labelspacing=0.5,
)

# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/dh_dt_scatter_maps.png", dpi=300)
fig.savefig("./figures/manuscript/dh_dt_scatter_maps.pdf", dpi=300)

### Scatter plots of $\Delta h$ and $\Delta t$ vs. various predictors

In [ ]:
plt.rcParams.update({"font.size": 10})
fig, ax = plt.subplots(2, 4, figsize=(8.5, 4.5))
ax = ax.flatten()

group_highlight = groups["globalS"]
marker_size = 30

# --------------------------------------------------------------------
# subplot data
ymax_top = 46
ymax_bottom = 85
subplots = [
    # a
    dict(
        x=analysis.res_hf_dymx_std,
        xlim=[0, 19],
        xlab=None,
        y=analysis[f"dh_{central_est_type}_{chronic_freq}_days"],
        ylim=[0, ymax_top],
        ylab=r"$\Delta h$ (cm)",
    ),
    # b
    dict(
        x=analysis.res_momn_q75_std,
        xlim=[0, 19],
        xlab=None,
        y=analysis[f"dh_{central_est_type}_{chronic_freq}_days"],
        ylim=[0, ymax_top],
    ),
    # c
    dict(
        x=analysis.tide_dymx_std,
        xlim=[0, 19],
        xlab=None,
        y=analysis[f"dh_{central_est_type}_{chronic_freq}_days"],
        ylim=[0, ymax_top],
    ),
    None,
    # d
    dict(
        x=analysis.res_hf_dymx_std,
        xlim=[0, 19],
        xlab="Storminess (cm)",
        # xlab=r"$\sigma\,$: daily max $\eta'_{hf}$ (cm)",
        y=analysis[f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"],
        ylim=[0, ymax_bottom],
        ylab=r"$\Delta t$ (years)",  # _{\mathrm{20}}$ (years)",
        # title="Storminess",
    ),
    # e
    dict(
        x=analysis.res_momn_q75_std,
        xlim=[0, 19],
        xlab="MSL variability (cm)",
        # xlab=r"$\sigma\,$: $\eta'_{lf}$ (cm)",
        y=analysis[f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"],
        ylim=[0, ymax_bottom],
        # title="Monthly MSL variability",
    ),
    # f
    dict(
        x=analysis.tide_dymx_std,
        xlim=[0, 19],
        xlab="High-tide modulation (cm)",
        # xlab=r"$\sigma\,$: daily max $\zeta$ (cm)",
        y=analysis[f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"],
        ylim=[0, ymax_bottom],
        # title="High-tide modulation",
    ),
    # f
    dict(
        x=analysis[f"slr_total_{scenario}_2020_2050"],
        xlim=[0, 40],
        xlab="Sea-level rise (cm)",  # "Int. SLR 2020–50 (cm)",
        # xlab=r"$\sigma\,$: daily max $\zeta$ (cm)",
        y=analysis[f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"],
        ylim=[0, ymax_bottom],
        # title="High-tide modulation",
        r_loc=(0.96, 0.89),
    ),
]

# --------------------------------------------------------------------

[a.remove() for a, sp in zip(ax, subplots) if sp is None]
subplots = [(a, sp) for a, sp in zip(ax, subplots) if sp is not None]
for k, (a, sp) in enumerate(subplots):
    o2c_scatter_plot(
        ax=a,
        x=sp["x"],
        y=sp["y"],
        ghl=group_highlight,
        col=[colors[n] for n in [0, 1]],
        ms=marker_size,
        title=sp["title"] if "title" in sp else None,
        splab=chr(97 + k),
        r_loc=sp["r_loc"] if "r_loc" in sp else None,
        xlim=sp["xlim"] if "xlim" in sp else None,
        xlab=sp["xlab"],
        ylim=sp["ylim"] if "ylim" in sp else None,
        ylab=sp["ylab"] if "ylab" in sp else None,
    )

# --------------------------------------------------------------------

# spacing trick to add space above top row of figures
# ax[0].annotate(
#     text=r"$~$",
#     xy=(0.5, 1.15),
#     xycoords="axes fraction",
# )

leglab = ["Global South", "Global North"]
fig.legend(
    leglab,
    ncols=1,
    loc="upper right",
    bbox_to_anchor=(0.95, 0.87),
    frameon=False,
    labelspacing=0.5,
)

# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/dh_dt_sigma_scatter_plots.png", dpi=300)
fig.savefig("./figures/manuscript/dh_dt_sigma_scatter_plots.pdf", dpi=300)

#### Correlations between predictors

In [ ]:
# correlations between the predictors
scn = scenario
# scn = "High"
quantities = [
    analysis[f"dh_{central_est_type}_{chronic_freq}_days"],
    analysis[f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020"],
    analysis.res_hf_dymx_std,
    analysis.res_momn_q75_std,
    analysis.tide_dymx_std,
    analysis[f"slr_total_{scn}_2020_2050"],
    analysis[f"slr_massdef_{scn}_2020_2050"],
    analysis[f"slr_ocean_dyn_{scn}_2020_2050"],
    analysis[f"slr_vlm_{scn}_2020_2050"],
]
names = [
    "dh",
    "dt",
    "Storminess",
    "MSL variability",
    "High-tide modulation",
    "Sea-level rise",
    "Mass/Deformation",
    "Sterodynamic",
    "Land motion",
]
rmat = pd.DataFrame(np.corrcoef(quantities, rowvar=True), index=names, columns=names)
z = np.isfinite(quantities[1])
r_dt = [np.corrcoef(quantities[1][z], q[z])[0, 1] for q in quantities]
rmat.loc["dt"] = r_dt
rmat["dt"] = r_dt
rmat

In [ ]:
# determine significance of correlations based on t-test
dof = 14  # degrees of freedom
tmat = rmat / np.sqrt((1 - rmat**2) / (dof))  # t-statistic
pvals = 2 * t.sf(np.abs(tmat), dof)
pvalmat = np.zeros_like(pvals).astype(str)
for m in range(len(names)):
    for n in range(len(names)):
        if m == n:
            pvalmat[m, n] = "-"
        elif pvals[m, n] <= 0.01:  # highly significant
            pvalmat[m, n] = f"**{pvals[m, n]:.3f}"
        elif pvals[m, n] <= 0.05:  # significant
            pvalmat[m, n] = f"*{pvals[m, n]:.3f}"
        else:
            pvalmat[m, n] = f"{pvals[m, n]:.3f}"
pvalmat = pd.DataFrame(pvalmat, index=names, columns=names)
pvalmat

#### Partial correlations

In [ ]:
def partial_correlation(x, y, Z):
    x = analysis[x].values  # variable 1
    y = analysis[y].values  # variable 2
    Z = analysis[Z].values  # confounding variables
    Z = np.hstack([Z, np.ones((Z.shape[0], 1))])
    kp = np.isfinite(x) & np.isfinite(y) & np.isfinite(Z).sum(axis=1).astype(bool)
    cy = np.linalg.lstsq(Z[kp, :], y[kp], rcond=None)[0]
    yp = (Z @ cy).flatten()
    cx = np.linalg.lstsq(Z[kp, :], x[kp], rcond=None)[0]
    xp = (Z @ cx).flatten()
    pc = np.corrcoef((y - yp)[kp], (x - xp)[kp])[0, 1]
    return pc


# partial correlations with dh controlling for a single confounding variable
prx = {
    "storminess": "res_hf_dymx_std",
    "MSL variability": "res_momn_q75_std",
    "high-tide modulation": "tide_dymx_std",
}
for c in prx:
    for v in [p for p in prx if p != c]:
        pc = partial_correlation(
            x=f"dh_{central_est_type}_{chronic_freq}_days",
            y=prx[v],
            Z=[prx[c]],
        )
        print(f"Partial correlation between dh and {v} controlling for {c}: {pc:.3f}")
v = "storminess"
c = [p for p in prx if p != v]
pc = partial_correlation(
    x=f"dh_{central_est_type}_{chronic_freq}_days",
    y=prx[v],
    Z=[prx[cn] for cn in c],
)
print(
    f"Partial correlation between dh and {v} controlling for {' and '.join(c)}: {pc:.3f}"
)

# # partial correlations with dt controlling for storminess and SLR
# scn = "Int"
# t0 = 2020
# pc = partial_correlation(
#     x=f"slr_total_{scn}_{t0}_{t0+30}",
#     y=f"dt_{central_est_type}_{chronic_freq}_days_{scn}_{t0}",
#     z=["res_hf_dymx_std"],
# )
# print(f"Partial correlation between dt and SLR controlling for storminess: {pc:.3f}")
# pc = partial_correlation(
#     x="res_hf_dymx_std",
#     y=f"dt_{central_est_type}_{chronic_freq}_days_{scn}_{t0}",
#     z=f"slr_total_{scn}_{t0}_{t0+30}",
# )
# print(f"Partial correlation between dt and storminess controlling for SLR: {pc:.3f}")

#### Variance explained in regression analysis

In [ ]:
def multiple_regression(y, factors):
    X = analysis[factors].values
    X = np.hstack([X, np.ones((X.shape[0], 1))])
    y = analysis[y].values
    z = np.isfinite(y)
    X = X[z, :]
    y = y[z]
    c = np.linalg.lstsq(X, y, rcond=None)[0]
    yp = X @ c
    r2 = 1 - np.sum((y - yp) ** 2) / np.sum((y - np.mean(y)) ** 2)
    return c, r2


# multiple regression of delta h onto the storminess proxy alone
c, r2 = multiple_regression(
    y=f"dh_{central_est_type}_{chronic_freq}_days",
    factors=["res_hf_dymx_std"],
)
print(f"Variance in dh explained by the regression onto storminess proxy: {r2:.2f}")

# multiple linear regression of the proxies on delta h
c, r2 = multiple_regression(
    y=f"dh_{central_est_type}_{chronic_freq}_days",
    factors=["res_hf_dymx_std", "res_momn_q75_std", "tide_dymx_std"],
)
print(f"Variance in dh explained by the regression onto all proxies: {r2:.2f}")

print("\n--------------------------------------------------\n")
for scn in ["IntLow", "Int", "High"]:
    for t0 in [2020, 2050]:

        # 17th, 50th, and 83rd percentiles of dt values
        dt_percentiles = analysis[
            f"dt_{central_est_type}_{chronic_freq}_days_{scn}_{t0}"
        ].quantile([0.17, 0.5, 0.83])
        print(
            f"17th, 50th, and 83rd percentiles of dt ({scn}, {t0}): {dt_percentiles.values}"
        )

        # multiple regression of delta t onto the storminess proxy alone
        c, r2 = multiple_regression(
            y=f"dt_{central_est_type}_{chronic_freq}_days_{scn}_{t0}",
            factors=["res_hf_dymx_std"],
        )
        print(
            f"Variance in dt ({scn}, {t0}) explained by the regression onto storminess proxy: {r2:.2f}"
        )

        # multiple regression of delta t onto SLR
        c, r2 = multiple_regression(
            y=f"dt_{central_est_type}_{chronic_freq}_days_{scn}_{t0}",
            factors=[f"slr_total_{scn}_{t0}_{t0+30}"],
        )
        print(
            f"Variance in dt ({scn}, {t0}) explained by the regression onto SLR: {r2:.2f}"
        )

        # multiple regression of  delta t onto the storminess proxy and SLR
        c, r2 = multiple_regression(
            y=f"dt_{central_est_type}_{chronic_freq}_days_{scn}_{t0}",
            factors=["res_hf_dymx_std", f"slr_total_{scn}_{t0}_{t0+30}"],
        )
        print(
            f"Variance in dt ({scn}, {t0}) explained by the regression onto storminess and SLR: {r2:.2f}"
        )
        print("\n--------------------------------------------------\n")

### Histogram violins: Global binary classifications

In [ ]:
violin_groups = [
    "lowLat",
    "highLat",
    # "highLat_eastHemi",
    # "highLat_westHemi",
    "island",
    "continent",
    "globalS",
    "globalN",
    # "globalS_island",
    # "not_globalS_island",
    # "globalS_lowLat_island",
    # "lucky",
]
violin_group_names = [
    "Low Latitude",
    "High Latitude",
    # "E.H. High Lat.",
    # "W.H. High Lat.",
    "Islands",
    "Continents",
    "Global South",
    "Global North",
    # "LLGS Islands",
    # "HLGN Continents",
]

subplots = [
    dict(
        var=f"dh_{central_est_type}_{chronic_freq}_days",
        mnmx_pctls=[0, 100],
        nbins=35,
        ylim=[0, 35],
        yticks=[0, 10, 20, 30],
        wfact=3.4,
        ylab=r"$\Delta h$ (cm)",
        leglab=True,
    ),
    dict(
        var=f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020",
        mnmx_pctls=[0, 100],
        nbins=25,
        ylim=[0, 70],
        yticks=[0, 20, 40, 60],
        wfact=6,
        ylab=r"$\Delta t$ (years)",
    ),
    dict(
        var="res_hf_dymx_std",
        mnmx_pctls=[0, 100],
        nbins=30,
        ylim=[0, 17.5],
        yticks=[0, 5, 10, 15],
        wfact=1.2,
        ylab=r"Storminess (cm)",
    ),
    dict(
        var="slr_total_Int_2020_2050",
        nbins=40,
        wfact=1.85,
        mnmx_pctls=[0, 100],
        ylim=[8, 27],
        yticks=[10, 15, 20, 25],
        ylab=r"Total relative SLR (cm)",
        leglab=True,
    ),
    # dict(
    #     var="res_momn_std",
    #     mnmx_pctls=[0, 100],
    #     nbins=15,
    #     ylim=[0, 17.5],
    #     yticks=[0, 5, 10, 15],
    #     wfact=1.15,
    #     ylab=r"Monthly MSL variability (cm)",
    # ),
    # dict(
    #     var="tide_dymx_std",
    #     mnmx_pctls=[0, 100],
    #     nbins=25,
    #     ylim=[0, 17.5],
    #     yticks=[0, 5, 10, 15],
    #     wfact=1.75,
    #     ylab=r"Tidal modulation (cm)",
    # ),
]

for sp in subplots:
    quantities = dict(
        samples={g: analysis[sp["var"]].loc[groups[g]].values for g in violin_groups},
        median=analysis[sp["var"]].median(),
        diff_sig=io_median_diff_percentiles.loc[sp["var"], significance_level],
    )
    sp.update(quantities)

In [ ]:
plt.rcParams.update({"font.size": 10})
fig, axes = plt.subplots(2, 2, figsize=(8.5, 6), sharex=True)
axes = axes.flatten()
[sp.update(dict(axis=axes[n])) for n, sp in enumerate(subplots)]
[ax.remove() for ax in axes[len(subplots) :]]

# set horizontal spacing of violin histograms
violin_locs = [0, 1, 3, 4, 6, 7]

Nvg = len(violin_groups)
for k, sp in enumerate(subplots):
    ax = sp["axis"]
    hist_violin(
        ax=ax,
        samples=sp["samples"],
        median=sp["median"],
        ylab=sp["ylab"],
        group_names=violin_group_names,
        sp_letter=chr(97 + k),
        width_factor=sp["wfact"],
        mnmx_pctls=sp["mnmx_pctls"],
        n_bins=sp["nbins"],
        ylim=sp["ylim"] if "ylim" in sp else None,
        yticks=sp["yticks"] if "yticks" in sp else None,
        violin_locs=violin_locs,
        diff_sig=sp["diff_sig"],
        leglab=sp["leglab"] if "leglab" in sp else None,
        colors=colors,
    )

# spacing trick to add space above top row of figures
axes[0].annotate(
    text=r"$~$",
    xy=(0.5, 1.25),
    xycoords="axes fraction",
)

leg_init = fig.legend()
ordered_leg_handles = [leg_init.legend_handles[k] for k in [3, 0, 1, 2]]
ordered_leg_labels = [h.get_label() for h in ordered_leg_handles]
ordered_leg_handles[1] = MulticolorPatch([colors[0], colors[3]])
leg_init.remove()
leg = fig.legend(
    handles=ordered_leg_handles,
    labels=ordered_leg_labels,
    ncols=2,
    loc="upper center",
    bbox_to_anchor=(0.53, 0.98),
    frameon=False,
    handler_map={MulticolorPatch: MulticolorPatchHandler()},
)


# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/dh_dt_group_comparisons.png", dpi=300)
fig.savefig("./figures/manuscript/dh_dt_group_comparisons.pdf", dpi=300)

### Histogram violins: Socioeconomic classifications

In [ ]:
violin_groups = [
    "globalS",
    "globalN_domestic",
    "globalS",
    "globalN_domestic_westHemi",
    "globalS",
    "globalN_domestic_NAEU",
]
violin_group_names = [
    "Global South",
    "Domestic G. North",
    "Global South",
    "W.H. Dom. G. North",
    "Global South",
    "Domestic N.A./Europe",
]

subplots = [
    dict(
        var=f"dh_{central_est_type}_{chronic_freq}_days",
        mnmx_pctls=[0, 100],
        nbins=35,
        ylim=[0, 35],
        yticks=[0, 10, 20, 30],
        wfact=3.4,
        ylab=r"$\Delta h$ (cm)",
        leglab=True,
    ),
    # dict(
    #     var=f"hdi",
    #     mnmx_pctls=[0, 100],
    #     nbins=15,
    #     ylim=[0.55, 1.05],
    #     yticks=[0.6, 0.8, 1.0],
    #     wfact=0.05,
    #     ylab=r"HDI",
    #     leglab=True,
    # ),
    dict(
        var=f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020",
        mnmx_pctls=[0, 100],
        nbins=25,
        ylim=[0, 70],
        yticks=[0, 20, 40, 60],
        wfact=6,
        ylab=r"$\Delta t$ (years)",
    ),
    dict(
        var="res_hf_dymx_std",
        mnmx_pctls=[0, 100],
        nbins=30,
        ylim=[0, 17.5],
        yticks=[0, 5, 10, 15],
        wfact=1.2,
        ylab=r"Storminess (cm)",
    ),
    # dict(
    #     var="slr_total_Int_2020_2050",
    #     nbins=40,
    #     wfact=1.85,
    #     mnmx_pctls=[0, 100],
    #     ylim=[8, 27],
    #     yticks=[10, 15, 20, 25],
    #     ylab=r"Sea-level rise (cm)",
    #     leglab=True,
    # ),
    dict(
        var="slr_massdef_Int_2020_2050",
        nbins=40,
        wfact=0.75,
        mnmx_pctls=[0, 100],
        ylim=[4, 12],
        yticks=[5, 7, 9, 11],
        ylab=r"Mass and deformation (cm)",
    ),
]

for sp in subplots:
    quantities = dict(
        samples={g: analysis[sp["var"]].loc[groups[g]].values for g in violin_groups},
        median=analysis[sp["var"]].median(),
        diff_sig=io_exclusion_median_diff_percentiles.loc[
            sp["var"], significance_level
        ],
    )
    sp.update(quantities)

In [ ]:
plt.rcParams.update({"font.size": 10})
fig, axes = plt.subplots(2, 2, figsize=(8, 6.5), sharex=True)
axes = axes.flatten()
[sp.update(dict(axis=axes[n])) for n, sp in enumerate(subplots)]
[ax.remove() for ax in axes[len(subplots) :]]

# set horizontal spacing of violin histograms
violin_locs = [0, 1, 3, 4, 6, 7]

Nvg = len(violin_groups)
for k, sp in enumerate(subplots):
    ax = sp["axis"]
    hist_violin(
        ax=ax,
        samples=sp["samples"],
        sample_names=violin_groups,
        median=sp["median"],
        ylab=sp["ylab"],
        group_names=violin_group_names,
        sp_letter=chr(97 + k),
        width_factor=sp["wfact"],
        mnmx_pctls=sp["mnmx_pctls"],
        n_bins=sp["nbins"],
        ylim=sp["ylim"] if "ylim" in sp else None,
        yticks=sp["yticks"] if "yticks" in sp else None,
        violin_locs=violin_locs,
        diff_sig=sp["diff_sig"],
        leglab=sp["leglab"] if "leglab" in sp else None,
        colors=colors,
    )

# spacing trick to add space above top row of figures
axes[0].annotate(
    text=r"$~$",
    xy=(0.5, 1.25),
    xycoords="axes fraction",
)

leg_init = fig.legend()
ordered_leg_handles = [leg_init.legend_handles[k] for k in [3, 0, 1, 2]]
ordered_leg_labels = [h.get_label() for h in ordered_leg_handles]
ordered_leg_handles[1] = MulticolorPatch([colors[0], colors[3]])
leg_init.remove()
leg = fig.legend(
    handles=ordered_leg_handles,
    labels=ordered_leg_labels,
    ncols=2,
    loc="upper center",
    bbox_to_anchor=(0.53, 0.98),
    frameon=False,
    handler_map={MulticolorPatch: MulticolorPatchHandler()},
)


# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/dh_dt_socioecon_comparisons.png", dpi=300)
fig.savefig("./figures/manuscript/dh_dt_socioecon_comparisons.pdf", dpi=300)

### Histogram violins: Contributions to SLR

In [ ]:
violin_groups = [
    "lowLat",
    "highLat",
    "highLat_eastHemi",
    "highLat_westHemi",
    "lowLat_eastHemi",
    "lowLat_westHemi",
    # "island",
    # "continent",
    # "globalS",
    # "globalN",
    # "globalS_island",
    # "not_globalS_island",
    # "globalS_lowLat_island",
    # "lucky",
]
violin_group_names = [
    "Low Latitude",
    "High Latitude",
    "E.H. High Lat.",
    "W.H. High Lat.",
    "E.H. Low Lat.",
    "W.H. Low Lat.",
    # "Islands",
    # "Continents",
    # "Global South",
    # "Global North",
    # "LLGS Islands",
    # "HLGN Continents",
]

subplots = [
    dict(
        var=f"dh_{central_est_type}_{chronic_freq}_days",
        mnmx_pctls=[0, 100],
        nbins=35,
        ylim=[0, 35],
        yticks=[0, 10, 20, 30],
        wfact=3.4,
        ylab=r"$\Delta h$ (cm)",
        leglab=True,
    ),
    dict(
        var=f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020",
        mnmx_pctls=[0, 100],
        nbins=25,
        ylim=[0, 70],
        yticks=[0, 20, 40, 60],
        wfact=6,
        ylab=r"$\Delta t$ (years)",
    ),
    dict(
        var="slr_total_Int_2020_2050",
        nbins=40,
        wfact=1.75,
        mnmx_pctls=[0, 100],
        ylim=[8, 27],
        yticks=[10, 15, 20, 25],
        ylab=r"Total relative SLR (cm)",
        leglab=True,
    ),
    dict(
        var="slr_massdef_Int_2020_2050",
        nbins=40,
        wfact=0.75,
        mnmx_pctls=[0, 100],
        ylim=[4, 12],
        yticks=[5, 7, 9, 11],
        ylab=r"Mass and deformation (cm)",
    ),
]

for sp in subplots:
    quantities = dict(
        samples={g: analysis[sp["var"]].loc[groups[g]].values for g in violin_groups},
        median=analysis[sp["var"]].median(),
        diff_sig=io_median_diff_percentiles.loc[sp["var"], significance_level],
    )
    sp.update(quantities)

In [ ]:
plt.rcParams.update({"font.size": 10})
fig, axes = plt.subplots(2, 2, figsize=(8, 6), sharex=True)
axes = axes.flatten()
[sp.update(dict(axis=axes[n])) for n, sp in enumerate(subplots)]

violin_locs = [0, 1, 3, 4, 6, 7]

Nvg = len(violin_groups)
for k, sp in enumerate(subplots):
    ax = sp["axis"]
    hist_violin(
        ax=ax,
        samples=sp["samples"],
        median=sp["median"],
        ylab=sp["ylab"],
        group_names=violin_group_names,
        sp_letter=chr(97 + k),
        width_factor=sp["wfact"],
        n_bins=sp["nbins"],
        mnmx_pctls=sp["mnmx_pctls"],
        violin_locs=violin_locs,
        diff_sig=sp["diff_sig"],
        leglab=sp["leglab"] if "leglab" in sp else None,
        colors=colors,
        ylim=sp["ylim"] if "ylim" in sp else None,
        yticks=sp["yticks"] if "yticks" in sp else None,
    )

# spacing trick to add space above top row of figures
axes[0].annotate(
    text=r"$~$",
    xy=(0.5, 1.25),
    xycoords="axes fraction",
)

leg_init = fig.legend()
ordered_leg_handles = [leg_init.legend_handles[k] for k in [3, 0, 1, 2]]
ordered_leg_labels = [h.get_label() for h in ordered_leg_handles]
ordered_leg_handles[1] = MulticolorPatch([colors[0], colors[3]])
leg_init.remove()
leg = fig.legend(
    handles=ordered_leg_handles,
    labels=ordered_leg_labels,
    ncols=2,
    loc="upper center",
    bbox_to_anchor=(0.53, 0.98),
    frameon=False,
    handler_map={MulticolorPatch: MulticolorPatchHandler()},
)

# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/slr_hemi_group_comparisons.png", dpi=300)
fig.savefig("./figures/manuscript/slr_hemi_group_comparisons.pdf", dpi=300)

### Histogram violins: Two socioeconomic groups

In [ ]:
print(f"There are {groups['globalS'].size} locations.")

Ngs_ex = np.sum(groups["globalS"]) - np.sum(groups["globalS_island"])
print(
    f"{Ngs_ex} of {np.sum(groups['globalS'])} locations are excluded from the Global South when forming the GS Islands group."
)
Ngn_ex = np.sum(groups["globalN"]) - np.sum(groups["globalN_continent"])
print(
    f"{Ngn_ex} of {np.sum(groups['globalN'])} locations are excluded from the Global North when forming the GN Continental group."
)

In [ ]:
# violin_groups = [
#     "globalS_island",
#     "globalN_continent",
# ]
# violin_group_names = [
#     "GS Islands",
#     "GN Continents",
# ]

# violin_groups = [
#     "globalS",
#     "globalN_westHemi_continent",
# ]
# violin_group_names = [
#     "GS",
#     "GN WH Continents",
# ]

violin_groups = [
    "globalS",
    "globalN_westHemi_highLat",
]
violin_group_names = [
    "Global South",
    "Global North, High-Latitude Western Hemisphere",
]

subplots = [
    dict(
        var=f"dh_{central_est_type}_{chronic_freq}_days",
        mnmx_pctls=[0, 100],
        nbins=35,
        ylim=[0, 35],
        yticks=[0, 10, 20, 30],
        wfact=3,
        ylab=r"$\Delta h$ (cm)",
        leglab=True,
    ),
    dict(
        var=f"dt_{central_est_type}_{chronic_freq}_days_{scenario}_2020",
        mnmx_pctls=[0, 100],
        nbins=25,
        ylim=[0, 70],
        yticks=[0, 20, 40, 60],
        wfact=6,
        ylab=r"$\Delta t$ (years)",
    ),
    dict(
        var="res_hf_dymx_std",
        mnmx_pctls=[0, 100],
        nbins=30,
        ylim=[0, 17.5],
        yticks=[0, 5, 10, 15],
        wfact=1.15,
        ylab=r"Storminess (cm)",
    ),
    # dict(
    #     var="slr_massdef_Int_2020_2050",
    #     nbins=40,
    #     wfact=0.75,
    #     mnmx_pctls=[0, 100],
    #     ylim=[4, 12],
    #     yticks=[5, 7, 9, 11],
    #     ylab=r"SLR: Mass and def. (cm)",
    # ),
    dict(
        var="slr_total_Int_2020_2050",
        nbins=40,
        wfact=1.75,
        mnmx_pctls=[0, 100],
        ylim=[8, 27],
        yticks=[10, 15, 20, 25],
        ylab=r"Sea-level rise (cm)",
        leglab=True,
    ),
    # dict(
    #     var="slr_total_IntHigh_2050_2080",
    #     nbins=40,
    #     wfact=1.75,
    #     mnmx_pctls=[0, 100],
    #     # ylim=[8, 27],
    #     yticks=[10, 15, 20, 25],
    #     ylab=r"Sea-level rise (cm)",
    #     leglab=True,
    # ),
]

for sp in subplots:
    quantities = dict(
        samples={g: analysis[sp["var"]].loc[groups[g]].values for g in violin_groups},
        median=analysis[sp["var"]].median(),
        diff_sig=io_exclusion_median_diff_percentiles.loc[
            sp["var"], significance_level
        ],
    )
    sp.update(quantities)

In [ ]:
plt.rcParams.update({"font.size": 10})
fig, axes = plt.subplots(1, 4, figsize=(8.5, 3), sharex=True)
axes = axes.flatten()
[sp.update(dict(axis=axes[n])) for n, sp in enumerate(subplots)]

violin_locs = [0, 1]

Nvg = len(violin_groups)
for k, sp in enumerate(subplots):
    ax = sp["axis"]
    hist_violin(
        ax=ax,
        samples=sp["samples"],
        median=sp["median"],
        ylab=sp["ylab"],
        group_names=violin_group_names,
        sp_letter=chr(97 + k),
        sp_letter_loc=(0.08, 0.88),
        width_factor=sp["wfact"],
        n_bins=sp["nbins"],
        mnmx_pctls=sp["mnmx_pctls"],
        violin_locs=violin_locs,
        diff_sig=sp["diff_sig"],
        leglab=sp["leglab"] if "leglab" in sp else None,
        colors=colors,
        ylim=sp["ylim"] if "ylim" in sp else None,
        yticks=sp["yticks"] if "yticks" in sp else None,
    )
    ax.set_xticklabels([])

# spacing trick to add space above/below figures
axes[0].annotate(
    text=r"$~$",
    xy=(0.5, 1.085),
    xycoords="axes fraction",
)
axes[0].annotate(
    text=r"$~$",
    xy=(0.5, -0.14),
    xycoords="axes fraction",
)

leg_init = fig.legend()

leg = fig.legend(
    handles=[leg_init.legend_handles[k] for k in [0, 1]],
    labels=violin_group_names,
    ncols=2,
    loc="upper center",
    bbox_to_anchor=(0.525, 0.99),
    frameon=False,
)

leg = fig.legend(
    handles=[leg_init.legend_handles[k] for k in [4, 2, 3]],
    ncols=3,
    loc="lower center",
    bbox_to_anchor=(0.525, 0.01),
    frameon=False,
)

leg_init.remove()

# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/dh_dt_2grp_socio_comparisons.png", dpi=300)
fig.savefig("./figures/manuscript/dh_dt_2grp_socio_comparisons.pdf", dpi=300)

### Histogram violins: Two groups SLR contributions

In [ ]:
# violin_groups = [
#     "globalS_island",
#     "globalN_continent",
# ]
# violin_group_names = [
#     "GS Islands",
#     "GN Continents",
# ]

# violin_groups = [
#     "globalS",
#     "globalN",
# ]
# violin_group_names = [
#     "Global South",
#     "Global North",
# ]

violin_groups = [
    "globalS",
    "globalN_westHemi_highLat",
]
violin_group_names = [
    "Global South",
    "Global North, High-Latitude Western Hemisphere",
]

subplots = [
    dict(
        var="slr_total_Int_2020_2050",
        nbins=40,
        wfact=1.75,
        mnmx_pctls=[0, 100],
        ylim=[8, 27],
        yticks=[10, 15, 20, 25],
        ylab=r"Total relative SLR (cm)",
        leglab=True,
    ),
    dict(
        var="slr_ocean_dyn_Int_2020_2050",
        nbins=20,
        wfact=0.8,
        mnmx_pctls=[0, 100],
        ylim=[3, 13],
        yticks=[4, 6, 8, 10, 12],
        ylab=r"Sterodynamic (cm)",
    ),
    dict(
        var="slr_massdef_Int_2020_2050",
        nbins=40,
        wfact=0.75,
        mnmx_pctls=[0, 100],
        ylim=[4, 12],
        yticks=[5, 7, 9, 11],
        ylab=r"Mass and deformation (cm)",
    ),
    dict(
        var="slr_vlm_Int_2020_2050",
        nbins=70,
        wfact=1.0,
        mnmx_pctls=[0, 100],
        ylim=[-5, 5],
        ylab=r"Vertical land motion (cm)",
    ),
]

for sp in subplots:
    quantities = dict(
        samples={g: analysis[sp["var"]].loc[groups[g]].values for g in violin_groups},
        median=analysis[sp["var"]].median(),
        diff_sig=io_exclusion_median_diff_percentiles.loc[
            sp["var"], significance_level
        ],
    )
    sp.update(quantities)

In [ ]:
plt.rcParams.update({"font.size": 10})
fig, axes = plt.subplots(1, 4, figsize=(8.5, 3), sharex=True)
axes = axes.flatten()
[sp.update(dict(axis=axes[n])) for n, sp in enumerate(subplots)]

violin_locs = [0, 1]

Nvg = len(violin_groups)
for k, sp in enumerate(subplots):
    ax = sp["axis"]
    hist_violin(
        ax=ax,
        samples=sp["samples"],
        median=sp["median"],
        ylab=sp["ylab"],
        group_names=violin_group_names,
        sp_letter=chr(97 + k),
        sp_letter_loc=(0.08, 0.88),
        width_factor=sp["wfact"],
        n_bins=sp["nbins"],
        mnmx_pctls=sp["mnmx_pctls"],
        violin_locs=violin_locs,
        diff_sig=sp["diff_sig"],
        leglab=sp["leglab"] if "leglab" in sp else None,
        colors=colors,
        ylim=sp["ylim"] if "ylim" in sp else None,
        yticks=sp["yticks"] if "yticks" in sp else None,
    )
    ax.set_xticklabels([])

# spacing trick to add space above/below figures
axes[0].annotate(
    text=r"$~$",
    xy=(0.5, 1.085),
    xycoords="axes fraction",
)
axes[0].annotate(
    text=r"$~$",
    xy=(0.5, -0.14),
    xycoords="axes fraction",
)

leg_init = fig.legend()

leg = fig.legend(
    handles=[leg_init.legend_handles[k] for k in [0, 1]],
    labels=violin_group_names,
    ncols=2,
    loc="upper center",
    bbox_to_anchor=(0.525, 0.99),
    frameon=False,
)

leg = fig.legend(
    handles=[leg_init.legend_handles[k] for k in [4, 2, 3]],
    ncols=3,
    loc="lower center",
    bbox_to_anchor=(0.525, 0.01),
    frameon=False,
)

leg_init.remove()

# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/slr_2grp_comparisons.png", dpi=300)
fig.savefig("./figures/manuscript/slr_2grp_comparisons.pdf", dpi=300)

### Mass and deformation pattern

In [ ]:
grd_prj = xr.open_dataset("./data/slr_scenarios/TR_gridded_projections.nc")
tot_slr = (
    grd_prj["rsl_total_Int"]
    .sel(percentiles=50)
    .sel(years=[2020, 2050])
    .diff(dim="years")
    .isel(years=0)
)
tot_mass_def = (
    xr.concat(
        [
            grd_prj[v].sel(percentiles=50)
            for v in [
                "rsl_landwaterstorage_Int",
                "rsl_AIS_Int",
                "rsl_GIS_Int",
                "rsl_glaciers_Int",
            ]
        ],
        dim="component",
    )
    .sum(dim="component")
    .sel(years=[2020, 2050])
    .diff(dim="years")
    .isel(years=0)
)
tot_mass_def

In [ ]:
fig, axes = plt.subplots(
    1,
    1,
    figsize=(8, 4),
    subplot_kw=dict(projection=ccrs.Mollweide(central_longitude=210)),
)


def slr_map(ax, slr_map, slr_tg, minmax, title):
    h = ax.imshow(
        np.roll(slr_map.values / 10, 180, axis=1),  # cm
        origin="lower",
        transform=ccrs.PlateCarree(),
        zorder=0,
        vmin=minmax[0],
        vmax=minmax[1],
        cmap="plasma",
    )
    ax.coastlines(zorder=15, lw=0.5)
    for g, mrkr in zip(["globalS", "globalN"], ["s", "o"]):
        ax.scatter(
            x=analysis.lon.loc[groups[g]],
            y=analysis.lat.loc[groups[g]],
            s=30,
            c=slr_tg.loc[groups[g]],
            vmin=minmax[0],
            vmax=minmax[1],
            marker=mrkr,
            lw=0.5,
            edgecolor="k",
            transform=ccrs.PlateCarree(),
            cmap="plasma",
            zorder=20,
        )
        ax.scatter(
            x=analysis.lon.loc[groups[g]],
            y=analysis.lat.loc[groups[g]],
            s=42,
            c="w",
            marker=mrkr,
            lw=0.5,
            edgecolor="w",
            transform=ccrs.PlateCarree(),
            zorder=19,
        )
    ax.add_feature(cfeature.LAND.with_scale("110m"), color="gray", zorder=10)
    ax.gridlines(linewidth=0.5, color="k", linestyle=":")
    axes.annotate(
        text=title,
        fontsize=11,
        xy=(0.5, 1.11),
        xycoords="axes fraction",
        ha="center",
    )
    axes.annotate(
        text=r"Intermediate Scenario, 2020-2050",
        fontsize=10,
        # fontstyle="italic",
        xy=(0.5, 1.05),
        xycoords="axes fraction",
        ha="center",
    )
    fig.colorbar(h, ax=ax, label="cm", shrink=0.8, pad=0.02, extend="both")


slr_map_input = [
    # dict(
    #     ax=axes[0],
    #     slr_map=tot_slr,
    #     slr_tg=analysis.slr_total_Int_2020_2050,
    #     minmax=[5, 21],
    #     title="Total Relative SLR",
    # ),
    dict(
        ax=axes[1] if type(axes) is np.ndarray else axes,
        slr_map=tot_mass_def,
        slr_tg=analysis.slr_massdef_Int_2020_2050,
        minmax=[0, 11],
        title="SLR Contribution from Mass and Deformation",
    ),
]
for input in slr_map_input:
    slr_map(
        input["ax"], input["slr_map"], input["slr_tg"], input["minmax"], input["title"]
    )

# spacing trick to add space above top row of figures
axes.annotate(
    text=r"$~$",
    xy=(0.5, 1.2),
    xycoords="axes fraction",
)
axes.annotate(
    text=r"$~$",
    xy=(0.5, -0.15),
    xycoords="axes fraction",
)

leglab = ["Global South", "Global North"]
# make dummy square and circle plots for the legend
att = dict(c="w", ls="none", lw=0.5, ms=7, mec="k", mew=0.5)
l1 = axes.plot(0, 1e7, marker="s", label=leglab[0], **att)[0]
l2 = axes.plot(0, 1e7, marker="o", label=leglab[1], **att)[0]
fig.legend(
    [l1, l2],
    leglab,
    ncols=2,
    loc="lower center",
    bbox_to_anchor=(0.42, -0.02),
    frameon=False,
    labelspacing=0.5,
)

# --------------------------------------------------------------------

fig.tight_layout()

# --------------------------------------------------------------------

fig.savefig("./figures/manuscript/massdef_map.png", dpi=300)
fig.savefig("./figures/manuscript/massdef_map.pdf", dpi=300)